In [ ]:
# 🔐 Система аутентификации - Пользователи и JWT

Добро пожаловать в интерактивный туториал по системе аутентификации! Вы научитесь работать с регистрацией, входом в систему и JWT токенами.

## 📚 Что вы изучите
- Регистрацию новых пользователей
- Аутентификацию и получение JWT токенов  
- Работу с защищенными endpoints
- Управление пользователями и ролями

## 🎯 Предварительные требования
- Завершенный [`01_project_overview.ipynb`](../quickstart/01_project_overview.ipynb)
- Запущенная база данных PostgreSQL
- Базовые знания HTTP и REST API

## ⏱️ Время прохождения
**15-20 минут**


In [ ]:
# Настройка окружения
import os
import sys
from pathlib import Path
import asyncio
from datetime import datetime

# Добавляем корневую папку проекта в путь
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

print(f"📁 Проект: {project_root}")
print(f"🕐 Время запуска: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Загрузка переменных окружения
from dotenv import load_dotenv
load_dotenv(project_root / '.env')

# Проверка критических настроек для аутентификации
auth_vars = ['SECRET_KEY', 'JWT_SECRET_KEY', 'DATABASE_URL']
print("\n🔒 Проверка настроек аутентификации:")
for var in auth_vars:
    value = os.getenv(var)
    status = "✅" if value else "❌" 
    print(f"{status} {var}: {'✓ настроено' if value else 'НЕ НАЙДЕНО'}")

if not all(os.getenv(var) for var in auth_vars):
    print("\n⚠️ Некоторые переменные не настроены! Убедитесь что .env файл создан.")


In [ ]:
## 🛠️ Импорт компонентов аутентификации

Давайте импортируем все необходимые компоненты для работы с аутентификацией:


In [ ]:
# Импорт компонентов аутентификации
try:
    # Основные сервисы
    from src.apps.auth.auth_service import AuthService
    from src.apps.auth.jwt_service import JWTService
    from src.apps.auth.password_service import PasswordService
    
    # Схемы данных
    from src.apps.users.schemas import UserCreate, UserLogin, UserResponse
    from src.apps.auth.schemas import LoginResponse, TokenPair
    
    # Модели
    from src.apps.users.models import User
    
    # Работа с БД
    from src.core.database import get_db, get_async_session
    
    print("✅ Все компоненты аутентификации импортированы успешно!")
    
    # Инициализация сервисов
    auth_service = AuthService()
    jwt_service = JWTService()
    password_service = PasswordService()
    
    print("🔧 Сервисы аутентификации инициализированы")
    
except ImportError as e:
    print(f"❌ Ошибка импорта: {e}")
    print("💡 Убедитесь что все зависимости установлены: pip install -r requirements.txt")


In [ ]:
## 👤 Интерактивная регистрация пользователя

Создадим интерактивную форму для регистрации нового пользователя:


In [ ]:
# Интерактивная форма регистрации
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import uuid

# Создаем виджеты формы
email_input = widgets.Text(
    value='demo@example.com',
    placeholder='Введите email',
    description='Email:',
    style={'description_width': 'initial'}
)

username_input = widgets.Text(
    value='demouser',
    placeholder='Введите username',
    description='Username:',
    style={'description_width': 'initial'}
)

full_name_input = widgets.Text(
    value='Demo User',
    placeholder='Введите полное имя',
    description='Полное имя:',
    style={'description_width': 'initial'}
)

password_input = widgets.Password(
    value='SecurePass123!',
    placeholder='Введите пароль',
    description='Пароль:',
    style={'description_width': 'initial'}
)

password_confirm_input = widgets.Password(
    value='SecurePass123!',
    placeholder='Подтвердите пароль',
    description='Повтор пароля:',
    style={'description_width': 'initial'}
)

register_button = widgets.Button(
    description='📝 Зарегистрировать',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

output = widgets.Output()

def validate_registration_data():
    """Валидация данных регистрации"""
    errors = []
    
    if not email_input.value or '@' not in email_input.value:
        errors.append("Email должен быть валидным")
    
    if not username_input.value or len(username_input.value) < 3:
        errors.append("Username должен быть минимум 3 символа")
    
    if len(password_input.value) < 8:
        errors.append("Пароль должен быть минимум 8 символов")
    
    if password_input.value != password_confirm_input.value:
        errors.append("Пароли не совпадают")
    
    return errors

def register_user(button):
    """Обработчик регистрации пользователя"""
    with output:
        clear_output()
        
        # Валидация
        errors = validate_registration_data()
        if errors:
            print("❌ Ошибки валидации:")
            for error in errors:
                print(f"  • {error}")
            return
        
        # Создание объекта пользователя (демо)
        user_data = {
            "id": str(uuid.uuid4()),
            "email": email_input.value,
            "username": username_input.value,
            "full_name": full_name_input.value,
            "is_active": True,
            "is_verified": False,
            "is_superuser": False,
            "created_at": datetime.now().isoformat()
        }
        
        print("✅ Пользователь успешно зарегистрирован!")
        print("📋 Данные пользователя:")
        print(json.dumps(user_data, indent=2, ensure_ascii=False))
        
        # Демонстрация хеширования пароля
        try:
            hashed_password = password_service.hash_password(password_input.value)
            print(f"\n🔒 Хеш пароля: {hashed_password[:50]}...")
            
            # Проверка пароля
            is_valid = password_service.verify_password(password_input.value, hashed_password)
            print(f"✅ Проверка пароля: {'Успешно' if is_valid else 'Ошибка'}")
            
        except Exception as e:
            print(f"⚠️ Ошибка хеширования: {e}")

register_button.on_click(register_user)

print("📝 Заполните форму регистрации:")
display(
    email_input, 
    username_input, 
    full_name_input, 
    password_input, 
    password_confirm_input,
    register_button,
    output
)
